In [1]:
from pickle import TRUE
import pandas as pd
import openstudio
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from datetime import timedelta
import email.utils as eutils
from statistics import mean
import time
import math
# import streamlit as st
import eppy as eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs
import PySimpleGUI as sg
# from PIL import Image, ImageTk
import os
from eppy.results import readhtml # the eppy module with functions to read the html
from eppy.results import fasthtml
import subprocess
import os
from os import system
import pylatex

from pylatex import Document, PageStyle, Head, MiniPage, Foot, LargeText, \
            MediumText, LineBreak, simple_page_number
from pylatex.utils import bold
from pylatex import Section, Subsection, Tabular, Math, TikZ, Axis, \
    Plot, Figure, Matrix, Alignat
from pylatex.utils import italic

In [55]:
# Appliance list
runCount = 2
Nbr = 2

runList = pd.read_csv("C:\REVIVE v24.1.0\Parametric Run List\PRL_2024-01-25.csv")
constructionList = pd.read_csv("C:\REVIVE v24.1.0\Databases\Construction Database.csv")

del appliance_list
#################
appliance_list = list(runList['APPLIANCE_LIST'][runCount].split(', '))

constructions = constructionList.shape[0]
for item in range(constructions):
    if str(constructionList['Name'][item]) in appliance_list:
        if 'FRIDGE' in constructionList['Name'][item]:
            fridge_demand = constructionList['Appliance_Rating'][item]
            fridge_cost = constructionList['Mechanical Cost'][item]
            fridge = (fridge_demand/(8760))*1000 # always on design load
        if 'DISHWASHER' in constructionList['Name'][item]:
            dishwasher_demand = constructionList['Appliance_Rating'][item]
            dishwasher_cost = constructionList['Mechanical Cost'][item]
            dishWasher = (((86.3 + (47.73 / (215 / dishwasher_demand)))/215) * ((88.4 + 34.9*Nbr)*(12/12))*(1/365)*1000)
        if 'CLOTHESWASHER' in constructionList['Name'][item]:
            clotheswasher_demand = constructionList['Appliance_Rating'][item]
            clotheswasher_cost = constructionList['Mechanical Cost'][item]
            clothesWasher = (clotheswasher_demand/365)*1000
        if 'CLOTHESDRYER' in constructionList['Name'][item]:
            clothesdryer_demand = constructionList['Appliance_Rating'][item]
            clothesdryer_cost = constructionList['Mechanical Cost'][item]
            clothesDryer = ((12.4*(164+46.5*Nbr)*1.18/3.01*(2.874/0.817-704/clothesdryer_demand)/(0.2184*(4.5*4.08+0.24)))/365)*1000
        if 'RANGE' in constructionList['Name'][item]:
            range_cost = constructionList['Mechanical Cost'][item]
        if 'LIGHTS' in constructionList['Name'][item]:
            fracHighEff = constructionList['Appliance_Rating'][item]
            lights_cost = constructionList['Mechanical Cost'][item]
try:
    fridge_demand
except:
    fridge_demand = 0
    fridge_cost = 0
    fridge = 0
try:
    dishwasher_demand
except:
    dishwasher_demand = 0
    dishwasher_cost = 0
    dishwasher = 0
try:
    clotheswasher_demand
except:
    clotheswasher_demand = 0
    clotheswasher_cost = 0
    clothesWasher = 0
try:
    clothesdryer_demand
except:
    clothesdryer_demand = 0
    clothesdryer_cost = 0
    clothesDryer = 0
try:
    range_cost
except:
    range_cost = 0
try:
    fracHighEff
except:
    fracHighEff = 0
    lights_cost = 0

        
total_appliance_cost = (fridge_cost + dishwasher_cost + clotheswasher_cost + clothesdryer_cost + range_cost)
print(appliance_list)
print(total_appliance_cost)


['CLOTHESWASHER 120', 'CLOTHESDRYER ES', 'RANGE', 'LIGHTS 1.0', 'DISHWASHER 269']
3221.0


In [47]:
fname = ("C:/Users/amitc_crl/OneDrive/Desktop/REVIVE v23.1.0/Testing/Name your batch of files_Morphed_Chicago-MDW_Phius Precriptive+Shd+DC_BA_eplustbl.htm")
filehandle = open(fname, 'r').read()
ltables = readhtml.lines_table(filehandle) # reads the tables with their titles

wallCostList = []
roofCostList = []
floorCostList = []
windowCostList = []
doorCostList = []
airSealingCostList = []
mechCostList = []
dhwCostList = []
applianceCostList = []



for ltable in ltables:
    if 'Construction Cost Estimate Summary' in '\n'.join(ltable[0]): #and 'For: Entire Facility' in '\n'.join(ltable[0]):
        firstCost = [(float(ltable[1][9][2])),0]
        firstCost = [(float(ltable[1][9][2])*1.8),0]
        print(firstCost)

    if 'Cost Line Item Details' in '\n'.join(ltable[0]):
        rows = len(ltable[1])
        for row in range(rows):
            # print(ltable[1][row][2])
            # print(ltable[1][row][5])
            if 'WALL' in str(ltable[1][row][2]):
                wallCostList.append(ltable[1][row][6])
            if 'ROOF' in str(ltable[1][row][2]):
                roofCostList.append(ltable[1][row][6])
            if 'FLOOR' in str(ltable[1][row][2]):
                floorCostList.append(ltable[1][row][6])
            if 'WINDOW' in str(ltable[1][row][2]):
                windowCostList.append(ltable[1][row][6])
            if 'DOOR' in str(ltable[1][row][2]):
                doorCostList.append(ltable[1][row][6])
            if 'AIR SEALING' in str(ltable[1][row][2]):
                airSealingCostList.append(ltable[1][row][6])
            if 'MECH' in str(ltable[1][row][2]):
                mechCostList.append(ltable[1][row][6])
            if 'DHW' in str(ltable[1][row][2]):
                dhwCostList.append(ltable[1][row][6])
            if 'APPLIANCE' in str(ltable[1][row][2]):
                applianceCostList.append(ltable[1][row][6])
            if 'PV COST' in str(ltable[1][row][2]):
                pvCost = (ltable[1][row][6])
            if 'BATTERY COST' in str(ltable[1][row][2]):
                batteryCost = (ltable[1][row][6])


wallCost = (sum(wallCostList))
roofCost = (sum(roofCostList))
floorCost = (sum(floorCostList))
airSealing = (sum(airSealingCostList))
mechCost = (sum(mechCostList))
dhwCost = (sum(dhwCostList))
applianceCost = (sum(applianceCostList))
pvCost = pvCost
batteryCost = batteryCost

print(batteryCost)

[121633.596, 0]
10338.91


In [53]:
IDF.setiddname("C:\EnergyPlusV9-5-0\Energy+.idd")
os.chdir("C:/Users/amitc_crl/OneDrive/Desktop/REVIVE v23.1.0/Testing")
open(str('test.idf'), 'w')
idf = IDF(str('test.idf'))

def constructionBuilder(idf, constructionName, constructionLayers):
        params = [x for x in constructionLayers]
        layers = {}
        count = 0
        for i,param in enumerate(params):
            count = count + 1
            layers['Layer_' + str(count)] = param
           
            

        layers.pop('Layer_1')
        idf.newidfobject('Construction',
        Name = str(constructionName),
        Outside_Layer = str(constructionLayers[0]),
        **layers)

constructionList = pd.read_csv("C:/Users/amitc_crl/OneDrive/Desktop/REVIVE v23.1.0/Databases/Construction Database.csv")

constructions = constructionList.shape[0]

for item in range(constructions):
    if str(constructionList['Outside_Layer'][item]) != 'nan':
        layers = [constructionList['Outside_Layer'][item],
        constructionList['Layer_2'][item],
        constructionList['Layer_3'][item],
        constructionList['Layer_4'][item],
        constructionList['Layer_5'][item],
        constructionList['Layer_6'][item],
        constructionList['Layer_7'][item],
        constructionList['Layer_8'][item],
        constructionList['Layer_9'][item],
        constructionList['Layer_10'][item]]

        layerList = [x for x in layers if str(x) != 'nan']

    

    constructionBuilder(idf, constructionList['Name'][item],layerList)


In [10]:
# MORA Deadly day analysis

df = pd.read_csv("C:/Users/amitc_crl/OneDrive/Desktop/REVIVE v23.1.0/Testing/Name your batch of files_Morphed_Chicago-MDW_BASE_BR_eplusout.csv")

heatingOutageStart = '01/26'
heatingOutageEnd = '02/02'
coolingOutageStart = '07/19'
coolingOutageEnd = '07/26'


df.rename(columns = {'Date/Time':'DateTime'}, inplace = True)
df[['Date2','Time']] = df.DateTime.str.split(expand=True)
df['Date'] = df['Date2'].map(str) + '/' + str(2020)
df['Time'] = (pd.to_numeric(df['Time'].str.split(':').str[0])-1).astype(str).apply(lambda x: f'0{x}' if len(x)==1 else x) + df['Time'].str[2:]
df['DateTime'] = df['Date'] + ' ' + df['Time']
df['DateTime'] = pd.to_datetime(df['DateTime'])

endWarmup = int((df[df['DateTime'] == '2020-01-01 00:00:00'].index.values))
dropWarmup = [*range(0, endWarmup,1)]

df = df.drop(index = dropWarmup)
df = df.reset_index()

df = df.drop(index = dropWarmup)
df = df.reset_index()

heatingOutageStart1 = datetime.strptime((str(heatingOutageStart) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=24)
coolingOutageStart1 = datetime.strptime((str(coolingOutageStart) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=24)
heatingOutageEnd1 = datetime.strptime((str(heatingOutageEnd) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=23)
coolingOutageEnd1 = datetime.strptime((str(coolingOutageEnd) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=23)

maskh = (df['DateTime'] >= heatingOutageStart1) & (df['DateTime'] <= heatingOutageEnd1)
maskc = (df['DateTime'] >= coolingOutageStart1) & (df['DateTime'] <= coolingOutageEnd1)

dfHeat = df.loc[maskh]
dfCool = df.loc[maskc]


MinDBOut = min(dfHeat['Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)'].tolist())
MinDPOut = min(dfHeat['Environment:Site Outdoor Air Dewpoint Temperature [C](Hourly)'].tolist())
MaxDBOut = max(dfCool['Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)'].tolist())
MaxDPOut = max(dfCool['Environment:Site Outdoor Air Dewpoint Temperature [C](Hourly)'].tolist())

print(MaxDBOut)
print(MaxDPOut)
print(MinDBOut)
print(MinDPOut)



41.91199247998387
26.90467837319528
-28.58992581008293
-32.05094858304751


In [13]:
path = "C:/Users/amitc_crl/OneDrive/Desktop/REVIVE v23.1.0/Parametric Runs"

os.chdir(path)
files = os.listdir(path)

runList = []

for file in files:
    if 'BR_eplusout.csv' in str(file):
        if 'P_TAM' in str(file):
            runList.append(file)

for case in runList:
    df = pd.read_csv(str(case))

    heatingOutageStart = '01/12'
    heatingOutageEnd = '01/18'
    coolingOutageStart = '07/14'
    coolingOutageEnd = '07/21'


    df.rename(columns = {'Date/Time':'DateTime'}, inplace = True)
    df[['Date2','Time']] = df.DateTime.str.split(expand=True)
    df['Date'] = df['Date2'].map(str) + '/' + str(2020)
    df['Time'] = (pd.to_numeric(df['Time'].str.split(':').str[0])-1).astype(str).apply(lambda x: f'0{x}' if len(x)==1 else x) + df['Time'].str[2:]
    df['DateTime'] = df['Date'] + ' ' + df['Time']
    df['DateTime'] = pd.to_datetime(df['DateTime'])

    endWarmup = int((df[df['DateTime'] == '2020-01-01 00:00:00'].index.values))
    dropWarmup = [*range(0, endWarmup,1)]

    df = df.drop(index = dropWarmup)
    df = df.reset_index()

    df = df.drop(index = dropWarmup)
    df = df.reset_index()

    heatingOutageStart1 = datetime.strptime((str(heatingOutageStart) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=24)
    coolingOutageStart1 = datetime.strptime((str(coolingOutageStart) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=24)
    heatingOutageEnd1 = datetime.strptime((str(heatingOutageEnd) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=23)
    coolingOutageEnd1 = datetime.strptime((str(coolingOutageEnd) + '/' + str(2020)), '%m/%d/%Y') + timedelta(hours=23)

    maskh = (df['DateTime'] >= heatingOutageStart1) & (df['DateTime'] <= heatingOutageEnd1)
    maskc = (df['DateTime'] >= coolingOutageStart1) & (df['DateTime'] <= coolingOutageEnd1)

    dfHeat = df.loc[maskh]
    dfCool = df.loc[maskc]

    RH = dfCool['ZONE 1:Zone Air Relative Humidity [%](Hourly)'].tolist()
    Temp = dfCool['ZONE 1:Zone Air Temperature [C](Hourly)'].tolist()

    RHdays = [RH[x:x+24] for x in range(0, len(RH), 24)]
    TEMPdays = [Temp[x:x+24] for x in range(0, len(Temp), 24)]

    avgRH = []
    avgTemp = []
    moraDays = []
    moraPF = []
    moraTotalDays = 0
    for day in range(7):
        avgRH.append(mean(RHdays[day]))
        avgTemp.append(mean(TEMPdays[day]))
        moraDays.append((49.593 - 48.580*np.array(mean(RHdays[day])*0.01) +25.887*np.array(mean(RHdays[day])*0.01)**2))
        moraPF.append(mean(TEMPdays[day])-((49.593 - 48.580*np.array(mean(RHdays[day])*0.01) +25.887*np.array(mean(RHdays[day])*0.01)**2)))
        if (mean(TEMPdays[day])-((49.593 - 48.580*np.array(mean(RHdays[day])*0.01) +25.887*np.array(mean(RHdays[day])*0.01)**2))) > 0:
            moraTotalDays = moraTotalDays+1


    # print(str(case) + '  ' + str(avgTemp))
    # print(str(case) + '  ' + str(avgRH))
    # print(str(case) + '  ' + str(moraDays))
    # print(str(case) + '  ' + str(moraPF))
    print(str(case) + '  ' + str(moraTotalDays))

P_TAM_M_TampaFL_BASE+Shd+DC_BR_eplusout.csv  0
P_TAM_M_TampaFL_BASE+Shd_BR_eplusout.csv  7
P_TAM_M_TampaFL_BASE_BR_eplusout.csv  7
P_TAM_M_TampaFL_IECC+0.4+Shd+DC_BR_eplusout.csv  0
P_TAM_M_TampaFL_IECC+0.4+Shd_BR_eplusout.csv  7
P_TAM_M_TampaFL_IECC+0.4_BR_eplusout.csv  7
P_TAM_M_TampaFL_IECC+Shd+DC_BR_eplusout.csv  0
P_TAM_M_TampaFL_IECC+Shd_BR_eplusout.csv  7
P_TAM_M_TampaFL_IECC_BR_eplusout.csv  7
P_TAM_M_TampaFL_Phius Prescriptive+Shd+DC_BR_eplusout.csv  0
P_TAM_M_TampaFL_Phius Prescriptive+Shd_BR_eplusout.csv  6
P_TAM_M_TampaFL_Phius Prescriptive_BR_eplusout.csv  6
P_TAM_TampaFL_BASE_BR_eplusout.csv  7
P_TAM_TampaFL_IECC+0.4_BR_eplusout.csv  6
P_TAM_TampaFL_IECC_BR_eplusout.csv  6
P_TAM_TampaFL_Phius Prescriptive_BR_eplusout.csv  5


In [88]:
carbonDatabase = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Carbon Correction Database.csv')
countryEmissionsDatabase = pd.read_csv("C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Country Emission Database.csv")
runList = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/ParametricRunList_2023-12-04_Short.csv')

carbonDatabase.head()

carbonMeasures = ['HP Replace 2', 'HP Replace 3', 'Vegan Bike Riding Carpenters']

emCO2 = []

for measure in range(carbonDatabase.shape[0]):
    if carbonDatabase['Name'][measure] in carbonMeasures:
        for country in range(countryEmissionsDatabase.shape[0]):
            if str(countryEmissionsDatabase['COUNTRY'][country]) == str(carbonDatabase['Country'][1]):
                ef = countryEmissionsDatabase['EF [kg/$]'][country]
            if str(countryEmissionsDatabase['COUNTRY'][country]) == 'USA':
                efUSA = countryEmissionsDatabase['EF [kg/$]'][country]
        emCO2.append((carbonDatabase['Cost'][measure]*ef*carbonDatabase['Labor Fraction'][measure], carbonDatabase['Year'][measure]))

firstCost = (15000,1)

# firstCost[0] = firstCost[0]*0.5


runCount = 1

for country in range(countryEmissionsDatabase.shape[0]):
                            if str(countryEmissionsDatabase['COUNTRY'][country]) == str(runList['ENVELOPE_COUNTRY'][runCount]):
                                ef = countryEmissionsDatabase['EF [kg/$]'][country]
                                # emCO2first = ((firstCost[0] * (1-runList['ENVELOPE_LABOR_FRACTION'][runCount]) * runList['ENVELOPE_COUNTRY'][runCount]) + 
                                #     (firstCost[0] * (runList['ENVELOPE_LABOR_FRACTION'][runCount]) * runList['ENVELOPE_COUNTRY'][runCount]))

                                # emCO2firstCost = [emCO2first,0]


# emCO2firstCost = [emCO2first,0]

emCO2.append(emCO2firstCost)
print(emCO2)
print(ef)
# print(emCO2first)


print(firstCost[0])
print(1-runList['ENVELOPE_LABOR_FRACTION'][runCount])


[(795.6000000000001, 40), (795.6000000000001, 60), (-3510.0, 1), [19500.0, 0]]
0.234
15000
0.6


In [30]:
carbonDatabase = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Carbon Correction Database.csv')

carbonDatabase.head()

carbonMeasures = ['HP Replace 2', 'HP Replace 3']

carbonMeasureCost = []
k_dirMR = 0.02


for measure in range(carbonDatabase.shape[0]):
    if carbonDatabase['Name'][measure] in carbonMeasures:
        carbonMeasureCost.append((carbonDatabase['Cost'][measure], carbonDatabase['Year'][measure]))

firstCost = (135000,0)
carbonMeasureCost.append(firstCost)
dirMR = carbonMeasureCost

c_dirMR = []
years = range(70)
for i in years:
        # c_dirMR = []
        year = i+1
        for row in dirMR:
            
            if row[1] == i:
                # print(row[1])
                c_dirMR.append(row[0]/((1+k_dirMR)**year))
            else:
                c_dirMR.append(0)

        print(c_dirMR)

# emCO2 = [(emCO2_firstCost,1),((8500*laborFraction*0.3),20),((8500*laborFraction*0.3),40),((8500*laborFraction*0.3),60)] 

[0, 0, 132352.94117647057]
[0, 0, 132352.94117647057, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [2]:
df = pd.read_csv("C:\Users\amitc_crl\OneDrive\Desktop\REVIVE v23.1.0\Parametric Runs\P_ELP_M_ElPasoTX_IECC+0.4+Shd_BR_eplusout.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (762130416.py, line 1)

In [166]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"

def costBuilder(idf, name, type, lineItemType, itemName, objEndUse, costEach, costArea):
    idf.newidfobject('ComponentCost:LineItem',
        Name = name,
        Type = type,
        Line_Item_Type = lineItemType,
        Item_Name = itemName,
        Object_EndUse_Key = objEndUse,
        Cost_per_Each = costEach,
        Cost_per_Area = costArea
        )
    
def glazingBuilder(idf, name, uFactor, shgc):
    idf.newidfobject('WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
        Name = name,
        UFactor = uFactor,
        Solar_Heat_Gain_Coefficient = shgc
        )



idf1 = IDF(fname)
windows = idf1.idfobjects['WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM']
# windowTable = pd.DataFrame(columns=['Name', 'U-Factor','SHGC'])
# for window in windows:
#     newResultRow = pd.DataFrame([{'Name':window.Name,
#                                   'U-Factor':window.UFactor,
#                                   'SHGC':window.Solar_Heat_Gain_Coefficient
#                                   }])
#     windowTable= pd.concat([windowTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# windowTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Window Database.csv')

# print(windows)

glazingSpecs = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Window Database.csv')

glazings = glazingSpecs.shape[0]

for item in range(glazings):
    glazingBuilder(idf1, glazingSpecs['Name'][item], glazingSpecs['U-Factor'][item],glazingSpecs['SHGC'][item])


In [156]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"


def constructionBuilder(idf, constructionName, constructionLayers):
    params = [x for x in constructionLayers]
    layers = {}
    count = 0
    for i,param in enumerate(params):
        count = count + 1
        layers['Layer_' + str(count)] = param

    layers.pop('Layer_1')
    idf.newidfobject('Construction',
    Name = str(constructionName),
    Outside_Layer = str(constructionLayers[0]),
    **layers)

# IDF.setiddname(iddfile)
idf1 = IDF(fname)



constructions = idf1.idfobjects['CONSTRUCTION']

# for construct in constructions:
#     newResultRow = pd.DataFrame([{'Name':construct.Name,
#                                 'Outside_Layer':construct.Outside_Layer, 
#                                 'Layer_2':construct.Layer_2, 
#                                 'Layer_3':construct.Layer_3, 
#                                 'Layer_4':construct.Layer_4, 
#                                 'Layer_5':construct.Layer_5, 
#                                 'Layer_6':construct.Layer_6,
#                                 'Layer_7':construct.Layer_7,
#                                 'Layer_8':construct.Layer_8,
#                                 'Layer_9':construct.Layer_9,
#                                 'Layer_10':construct.Layer_10}])
#     ConstructionTable= pd.concat([ConstructionTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# ConstructionTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Construction Database.csv')

# constructionList = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Construction Database.csv')

# constructions = constructionList.shape[0]

# for item in range(constructions):
    
#     layers = [constructionList['Outside_Layer'][item],
#     constructionList['Layer_2'][item],
#     constructionList['Layer_3'][item],
#     constructionList['Layer_4'][item],
#     constructionList['Layer_5'][item],
#     constructionList['Layer_6'][item],
#     constructionList['Layer_7'][item],
#     constructionList['Layer_8'][item],
#     constructionList['Layer_9'][item],
#     constructionList['Layer_10'][item]]

#     layerList = [x for x in layers if str(x) != 'nan']

#     constructionBuilder(idf1, constructionList['Name'][item],layerList)

# IDFconstructions = idf1.idfobjects['CONSTRUCTION']

# print(IDFconstructions)


[
CONSTRUCTION,
    Brick Wall,               !- Name
    M01 100mm brick;          !- Outside Layer
, 
CONSTRUCTION,
    Ext_Door1,                !- Name
    G05 25mm wood;            !- Outside Layer
, 
CONSTRUCTION,
    Thermal Mass,             !- Name
    G05 25mm wood;            !- Outside Layer
, 
CONSTRUCTION,
    Interior Floor,           !- Name
    Plywood (Douglas Fir) - 12.7mm,    !- Outside Layer
    F05 Ceiling air space resistance,    !- Layer 2
    G01a 19mm gypsum board;    !- Layer 3
, 
CONSTRUCTION,
    Exterior Slab UnIns,      !- Name
    M15 200mm heavyweight concrete;    !- Outside Layer
, 
CONSTRUCTION,
    Exterior Slab + 2in EPS,    !- Name
    EPS 2in,                  !- Outside Layer
    M15 200mm heavyweight concrete;    !- Layer 2
, 
CONSTRUCTION,
    Exterior Wall,            !- Name
    F11 Wood siding,          !- Outside Layer
    R-11 3.5in Wood Stud,     !- Layer 2
    G01a 19mm gypsum board;    !- Layer 3
, 
CONSTRUCTION,
    Interior Wall,     

In [138]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"

# IDF.setiddname(iddfile)
idf1 = IDF(fname)

def materialBuilder(idf, name, rough, thick, conduct, dense, heatCap):
    idf.newidfobject('Material',
        Name = str(name),
        Roughness = str(rough),
        Thickness = thick,
        Conductivity = conduct,
        Density = dense,
        Specific_Heat = heatCap
        )

# MaterialTable = pd.DataFrame(columns=['Name', 'Roughness', 'Thickness', 'Conductivity', 'Density', 'Specific_Heat'])
# materials = idf1.idfobjects["MATERIAL"]
# for material in materials:
#     newResultRow = pd.DataFrame([{'Name':material.Name,
#                                   'Roughness':material.Roughness,
#                                   'Thickness': material.Thickness,
#                                   'Conductivity':material.Conductivity,
#                                   'Density':material.Density,
#                                   'Specific_Heat': material.Specific_Heat}])
#     MaterialTable = pd.concat([MaterialTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# MaterialTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Material_Database.csv')

# materials = pd.read_csv("C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Material_Database.csv")
# materialList = materials.shape[0]

# for item in range(materialList):
#     materialBuilder(idf1, materials['Name'][item], materials['Roughness'][item], materials['Thickness'][item], materials['Conductivity'][item],
#                     materials['Density'][item], materials['Specific_Heat'][item])